In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np

tfe.enable_eager_execution()

In [2]:
N, D, W, H = 10, 20, 30, 40
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3, '<START>': 4, '<END>': 5}
V = len(word_to_idx)
T = 13

features = np.linspace(-0.5, 1.7, num=N*D).reshape(N, D).astype(np.float32)
captions = (np.arange(N * T) % V).reshape(N, T).astype(np.int32)

input_features = tf.keras.layers.Input(shape=(D,), dtype=tf.float32)
project_layer = tf.keras.layers.Dense(units=H)
h0 = project_layer(input_features)

input_captions = tf.keras.layers.Input(shape=(None, T-1), dtype=tf.int32)
w_embedding = tf.keras.layers.Embedding(V, W, input_length=T-1, embeddings_initializer=tf.keras.initializers.RandomNormal())
output_x = w_embedding(input_captions)

input_state_c = tf.keras.layers.Input(shape=(H,), dtype=tf.float32)
decoder_initial_state = [h0, input_state_c]
decoder = tf.keras.layers.CuDNNLSTM(units=H, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(output_x, initial_state=decoder_initial_state)

dense_layer = tf.keras.layers.Dense(units=V, activation='softmax')
scores = dense_layer(decoder_outputs)
# print(captions[:, 1:].shape)  # shpae = (10, 12)
# tf.one_hot(captions[:, 1:], depth=V)

captioning_model = tf.keras.Model(inputs=[input_features, input_captions, input_state_c], outputs=scores)
captioning_model.compile(optimizer=tf.train.RMSPropOptimizer(0.005), loss='categorical_crossentropy')
captioning_model.fit(x=[features, captions[:, :-1], np.zeros((features.shape[0], H), dtype=np.float32)], 
                     y=tf.one_hot(captions[:, 1:], depth=V, dtype=tf.float32), epochs=120)

Epoch 1/120
1/1 [==============================] - 1s 898ms/step - loss: 1.6069
Epoch 2/120
1/1 [==============================] - 0s 46ms/step - loss: 1.6069
Epoch 3/120
1/1 [==============================] - 0s 28ms/step - loss: 1.6069
Epoch 4/120
1/1 [==============================] - 0s 28ms/step - loss: 1.6068
Epoch 5/120
1/1 [==============================] - 0s 26ms/step - loss: 1.6068
Epoch 6/120
1/1 [==============================] - 0s 26ms/step - loss: 1.6067
Epoch 7/120
1/1 [==============================] - 0s 27ms/step - loss: 1.6067
Epoch 8/120
1/1 [==============================] - 0s 27ms/step - loss: 1.6067
Epoch 9/120
1/1 [==============================] - 0s 27ms/step - loss: 1.6066
Epoch 10/120
1/1 [==============================] - 0s 26ms/step - loss: 1.6066
Epoch 11/120
1/1 [==============================] - 0s 28ms/step - loss: 1.6065
Epoch 12/120
1/1 [==============================] - 0s 27ms/step - loss: 1.6065
Epoch 13/120
1/1 [==============================

Epoch 104/120
1/1 [==============================] - 0s 26ms/step - loss: 1.3212
Epoch 105/120
1/1 [==============================] - 0s 25ms/step - loss: 1.2881
Epoch 106/120
1/1 [==============================] - 0s 25ms/step - loss: 1.2513
Epoch 107/120
1/1 [==============================] - 0s 25ms/step - loss: 1.2107
Epoch 108/120
1/1 [==============================] - 0s 24ms/step - loss: 1.1661
Epoch 109/120
1/1 [==============================] - 0s 25ms/step - loss: 1.1177
Epoch 110/120
1/1 [==============================] - 0s 26ms/step - loss: 1.0657
Epoch 111/120
1/1 [==============================] - 0s 24ms/step - loss: 1.0104
Epoch 112/120
1/1 [==============================] - 0s 24ms/step - loss: 0.9526
Epoch 113/120
1/1 [==============================] - 0s 24ms/step - loss: 0.8929
Epoch 114/120
1/1 [==============================] - 0s 24ms/step - loss: 0.8321
Epoch 115/120
1/1 [==============================] - 0s 24ms/step - loss: 0.7712
Epoch 116/120
1/1 [=========

In [93]:
input_start = tf.keras.layers.Input(shape=(None, 1), dtype=tf.int32)
output_start = w_embedding(input_start)

sampling_w_embeded_model = tf.keras.Model(inputs=input_start, outputs=output_start)
sampling_proj_model = tf.keras.Model(inputs=input_features, outputs=h0)

input_state_h = tf.keras.layers.Input(shape=(H,), dtype=tf.float32)
sampling_initial_state = [input_state_h, input_state_c]

sampling_initial_state
sampling_lstm_input = tf.keras.layers.Input(shape=(None, W))
sampling_lstm_output, sampling_h, sampling_c = decoder(sampling_lstm_input, initial_state=sampling_initial_state)
sampling_output_state = [sampling_h, sampling_c]
sampling_dense_output = dense_layer(sampling_lstm_output)

sampling_decoder = tf.keras.Model(inputs=[sampling_lstm_input]+sampling_initial_state, outputs=[sampling_dense_output]+sampling_output_state)

In [99]:
def sampling_caption(input_features, max_length=30):
    N = features.shape[0]
    
    start_embed = tf.convert_to_tensor(word_to_idx['<START>'] * np.ones(N, dtype=np.int32))
    input_x = sampling_w_embeded_model.predict(start_embed)
    initial_h = sampling_proj_model.predict(input_features)
    
    input_state = [initial_h, np.zeros_like(initial_h)]
    
    captions = (word_to_idx['<NULL>'] * np.ones((N, max_length))).astype(np.int32)
    
    # cur_dense_output, cur_state_h, cur_state_c = sampling_decoder.predict([input_x]+input_state)
    # print(cur_state_h)
    # print(input_x.shape)
    
    for t in range(max_length):
        cur_dense_output, cur_state_h, cur_state_c = sampling_decoder.predict([input_x]+input_state)
        
        # print("the shape is")
        # print(captions[:, t].shape)
        captions[:, t] += tf.squeeze(tf.cast(tf.argmax(cur_dense_output, axis=2), tf.int32))
        input_x = sampling_w_embeded_model.predict(captions[:, t])
        input_state = [cur_state_h, cur_state_c]
    
    return captions

In [100]:
sampling_caption(features).dtype

captions dtype
int32


dtype('int32')

In [5]:
x = np.asarray([[0, 3, 1, 2], [2, 1, 0, 3]])
input_x = tf.keras.layers.Input(shape=(None, 4))
w_embedding = tf.keras.layers.Embedding(5, 3, input_length=T, embeddings_initializer=tf.keras.initializers.RandomNormal())
# out = w_embedding(input_x)
# model = tf.keras.Model(inputs=[input_x], outputs=out)
# model.apply(tf.convert_to_tensor(x, dtype=tf.int32))
w_embedding(tf.convert_to_tensor(x, dtype=tf.int32))

<tf.Tensor: id=54130, shape=(2, 4, 3), dtype=float32, numpy=
array([[[ 0.09987143, -0.03769374, -0.03912736],
        [ 0.02454221,  0.06779458,  0.06717668],
        [-0.01099924, -0.06941702, -0.03229584],
        [ 0.00574606, -0.08676705, -0.01706604]],

       [[ 0.00574606, -0.08676705, -0.01706604],
        [-0.01099924, -0.06941702, -0.03229584],
        [ 0.09987143, -0.03769374, -0.03912736],
        [ 0.02454221,  0.06779458,  0.06717668]]], dtype=float32)>